In [3]:
import tensorflow as tf
import time
import numpy as np
import os
import copy
import pickle
import argparse
import utility
import pandas as pd
from sklearn.metrics import *

2023-03-28 11:32:31.133344: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-28 11:32:31.195794: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
def loadmodel(self, saver, checkpoint_dir):
    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt and ckpt.model_checkpoint_path:
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
        return True
    else:
        return False

In [5]:
reader = tf.train.load_checkpoint('ml1m-6/BPR_check_points_2')
shape_from_key = reader.get_variable_to_shape_map()
dtype_from_key = reader.get_variable_to_dtype_map()

sorted(shape_from_key.keys())


['BPR/P', 'BPR/Q']

In [6]:
reader.get_tensor('BPR/P')

array([[ 1.43514667e-03, -3.01657477e-03, -1.71056297e-03, ...,
        -1.28840026e-03,  2.55405903e-03, -3.80083174e-03],
       [-4.36936622e-04,  2.04426404e-02,  1.68757234e-02, ...,
         2.37677014e-03,  3.48560698e-03,  2.24851351e-03],
       [-1.61453262e-02,  5.16602723e-03, -4.78330208e-03, ...,
         1.67072657e-02,  1.58163588e-02,  1.61972940e-02],
       ...,
       [-7.28358600e-07,  1.48044619e-05, -2.71302679e-06, ...,
         2.22814106e-06,  1.15002968e-06,  2.26932098e-05],
       [ 2.21450254e-02, -2.41685845e-02, -1.28706237e-02, ...,
        -2.19115447e-02,  1.06267761e-02, -2.11809929e-02],
       [ 2.25834027e-02, -1.46012595e-02, -4.17736545e-02, ...,
        -1.62176639e-02,  6.48175478e-02,  3.38744298e-02]], dtype=float32)

In [7]:
def print_checkpoint(save_path):
    reader = tf.train.load_checkpoint(save_path)
    shapes = reader.get_variable_to_shape_map()
    dtypes = reader.get_variable_to_dtype_map()
    print(f"Checkpoint at '{save_path}':")
    for key in shapes:
        print(f"  (key='{key}', shape={shapes[key]}, dtype={dtypes[key].name}, "
            f"value={reader.get_tensor(key)})")


In [8]:
reader.get_tensor('BPR/Q').shape

(1481, 20)

In [9]:
np.save('Q_tensor.npy', reader.get_tensor('BPR/Q')) # save
np.save('P_tensor.npy', reader.get_tensor('BPR/P')) # save
# new_num_arr = np.load('Q_tensor.npy') # load

In [12]:
print_checkpoint('ml1m-6/BPR_check_points_2')

Checkpoint at 'ml1m-6/BPR_check_points_2':
  (key='BPR/Q', shape=[1481, 20], dtype=float32, value=[[-0.01458513 -0.0278788  -0.02968836 ...  0.01007636  0.01964535
   0.01039099]
 [ 0.02178593 -0.0296652  -0.00069894 ... -0.02580264  0.00797478
  -0.03780304]
 [ 0.01837691 -0.03319061 -0.01558379 ... -0.02123228  0.00159044
  -0.02819812]
 ...
 [ 0.00656306 -0.00753659  0.00897468 ... -0.00777141 -0.02740526
  -0.03508031]
 [-0.00663131 -0.00444541 -0.00344932 ...  0.00363906  0.00850748
  -0.00016008]
 [-0.0069091  -0.01007038 -0.00190771 ...  0.00596577  0.01874839
   0.00922376]])
  (key='BPR/P', shape=[6036, 20], dtype=float32, value=[[ 1.43514667e-03 -3.01657477e-03 -1.71056297e-03 ... -1.28840026e-03
   2.55405903e-03 -3.80083174e-03]
 [-4.36936622e-04  2.04426404e-02  1.68757234e-02 ...  2.37677014e-03
   3.48560698e-03  2.24851351e-03]
 [-1.61453262e-02  5.16602723e-03 -4.78330208e-03 ...  1.67072657e-02
   1.58163588e-02  1.61972940e-02]
 ...
 [-7.28358600e-07  1.48044619e-05 

In [13]:
Rec = np.matmul(reader.get_tensor('BPR/P'), reader.get_tensor('BPR/Q').T)

In [14]:
Rec

array([[ 0.14318259,  0.1976434 ,  0.45083743, ..., -0.23172706,
        -0.24774326, -0.23741563],
       [ 0.26357174,  0.38300318,  0.82533073, ..., -0.41900152,
        -0.45119196, -0.43492287],
       [ 0.23676388,  0.31873655,  0.7309779 , ..., -0.3834138 ,
        -0.40430233, -0.38593838],
       ...,
       [ 0.04913295,  0.06770388,  0.15442331, ..., -0.07954081,
        -0.08493978, -0.08139199],
       [ 0.2710443 ,  0.4062181 ,  0.86440605, ..., -0.427136  ,
        -0.4662867 , -0.45132524],
       [ 0.29020768,  0.41033268,  0.90996706, ..., -0.46618685,
        -0.4960658 , -0.47551146]], dtype=float32)

In [15]:
train_df = pd.read_pickle(r'ml1m-6/training_df.pkl')
vali_df = pd.read_pickle(r'ml1m-6/testing_df.pkl')   # for validation
# vali_df = pickle.load(open('./' + dataname + '/testing_df.pkl'))  # for testing
key_genre = pd.read_pickle(r'ml1m-6/key_genre.pkl')
item_idd_genre_list = pd.read_pickle(r'ml1m-6/item_idd_genre_list.pkl')
genre_item_vector = pd.read_pickle(r'ml1m-6/genre_item_vector.pkl')
genre_count = pd.read_pickle(r'ml1m-6/genre_count.pkl')
user_genre_count = pd.read_pickle(r'ml1m-6/user_genre_count.pkl')

num_item = len(train_df['item_id'].unique())
num_user = len(train_df['user_id'].unique())
num_genre = len(key_genre)


In [16]:
item_genre_list = []
for u in range(num_item):
    gl = item_idd_genre_list[u]
    tmp = []
    for g in gl:
        if g in key_genre:
            tmp.append(g)
    item_genre_list.append(tmp)

In [17]:
utility.ranking_analysis(Rec, vali_df, train_df, key_genre, item_genre_list,
                                                      user_genre_count)


####################################################################################################
# System-level Recall:
# 			Recall@1	Recall@5	Recall@10	Recall@15
# Sci-Fi		0.04175		0.13105		0.18754		0.22871
# Horror		0.00000		0.00916		0.03272		0.05549
# Crime		0.00013		0.01166		0.06871		0.12237
# Adventure		0.04082		0.12683		0.16713		0.19502
# Children's		0.00000		0.00520		0.02953		0.05582
# Romance		0.00402		0.03643		0.05829		0.07856
# relative std		1.31643		1.01823		0.69560		0.55100
####################################################################################################
# User-level Recall:
# 			Recall@1	Recall@5	Recall@10	Recall@15
# Sci-Fi		0.05482		0.16803		0.24313		0.28622
# Horror		0.00000		0.00851		0.03932		0.07954
# Crime		0.00004		0.00981		0.06694		0.13382
# Adventure		0.05448		0.16031		0.21845		0.24103
# Children's		0.00000		0.00607		0.03618		0.06613
# Romance		0.00472		0.05298		0.08052		0.10498
# relative std		1.32852		1.03720		0.73815		0.54473
###########

(array([1.37345983, 1.19663683, 0.98420816, 0.79713475]),
 array([1.3164328 , 1.01822575, 0.69559528, 0.55100164]))

In [52]:
Q_tensor = (np.load('Q_tensor.npy'))
P_tensor = (np.load('P_tensor.npy'))
Rec = np.matmul(P_tensor, Q_tensor.T)

In [53]:
utility.ranking_analysis(Rec, vali_df, train_df, key_genre, item_genre_list,
                                                      user_genre_count)


####################################################################################################
# System-level Recall:
# 			Recall@1	Recall@5	Recall@10	Recall@15
# Sci-Fi		0.02311		0.09894		0.17535		0.23561
# Horror		0.01374		0.05641		0.10064		0.13990
# Crime		0.01818		0.08024		0.13867		0.19032
# Adventure		0.02120		0.09090		0.15931		0.21637
# Children's		0.01420		0.05737		0.10250		0.14651
# Romance		0.01402		0.06079		0.11284		0.15781
# relative std		0.21325		0.22761		0.21699		0.19829
####################################################################################################
# User-level Recall:
# 			Recall@1	Recall@5	Recall@10	Recall@15
# Sci-Fi		0.03001		0.11798		0.20318		0.26870
# Horror		0.01442		0.05783		0.10390		0.15012
# Crime		0.01913		0.08711		0.14762		0.20453
# Adventure		0.02851		0.11374		0.18733		0.24986
# Children's		0.01513		0.05909		0.10926		0.15490
# Romance		0.01881		0.07423		0.13453		0.18373
# relative std		0.29072		0.28184		0.25068		0.22132
###########

(array([0.47163318, 0.46831565, 0.45548423, 0.4395613 ]),
 array([0.2132465 , 0.22760523, 0.21698537, 0.19828977]))

In [54]:
utility.test_model_all(Rec, vali_df, train_df)

precision_1	[0.1598434],	||	 precision_5	[0.1363475],	||	 precision_10	[0.1221787],	||	 precision_15	[0.1118548]
recall_1   	[0.0272944],	||	 recall_5   	[0.1075401],	||	 recall_10   	[0.1816718],	||	 recall_15   	[0.2408833]
f_measure_1	[0.0466270],	||	 f_measure_5	[0.1202425],	||	 f_measure_10	[0.1461010],	||	 f_measure_15	[0.1527703]
ndcg_1     	[0.1598434],	||	 ndcg_5     	[0.1607376],	||	 ndcg_10     	[0.1780276],	||	 ndcg_15     	[0.1951911]


(array([0.15984336, 0.13634745, 0.12217871, 0.11185475]),
 array([0.02729443, 0.10754011, 0.1816718 , 0.2408833 ]),
 [0.04662695945850442,
  0.12024246082827848,
  0.14610096393529334,
  0.15277025982451498],
 array([0.15984336, 0.16073764, 0.1780276 , 0.19519114]))